# Polarities

In [ ]:
import pandas as pd
df = pd.read_csv(r'./richtig.csv', delimiter=";", error_bad_lines=False, engine ='python')

In [30]:
import re
import string
from germansentiment import SentimentModel

#we used Sentimentmodel to create rows: polarity_negative, polarity_neutral and polarity_positive
#first it was created for all religions (richtig2.csv) then on all ethnics (richtig_polarities.csv)

#model used to get the sentiments/polarities
model = SentimentModel('mdraw/german-news-sentiment-bert')
#function to create the different polarities of a pandas dataframe by iterating over every row
def create_polarities(pandas):
    for row in pandas.itertuples(index=True, name='Pandas'):
        try:
            cleanedString = re.sub(r'[\[\]\'\,]', '', row.tokens_suggestions_cleaned_root).split()
            polarities = model.predict_sentiment(cleanedString)
            counter = 0
            polarity_negative = []
            polarity_neutral = []
            polarity_positive = []
            for polarity in polarities:
                if polarity == 'negative':
                    polarity_negative.append(cleanedString[counter])
                elif polarity == 'neutral':
                    polarity_neutral.append(cleanedString[counter])
                elif polarity == 'positive':
                    polarity_positive.append(cleanedString[counter])
                counter += 1
            pandas.at[row.Index, 'polarity_negative'] = polarity_negative
            pandas.at[row.Index, 'polarity_neutral'] = polarity_neutral
            pandas.at[row.Index, 'polarity_positive'] = polarity_positive
        except IndexError:
            continue;
            
#remove every not relevant row (row.tokensroot not in terms) of the dataframe 
terms = ['atheismus', 'christentum', 'islam', 'judentum', 'evangelismus', 'katholizismus']
filterIndex = []
chars = re.escape(string.punctuation)
for row in df.itertuples(index=True, name='Pandas'):
    cleanedString = row.tokensroot.strip("[]',").replace(r",", '').replace(r"'", '').split()
    for tokensroot in cleanedString:
        if tokensroot not in terms and row.Index not in filterIndex:
            filterIndex.append(row.Index)
            break;
df2 = df.drop(filterIndex)

#append the polarities to the dataframe and then run the function above to get the polarities
df['polarity_negative'] = ''
df['polarity_neutral'] = ''
df['polarity_positive'] = ''
df2['polarity_negative'] = ''
df2['polarity_neutral'] = ''
df2['polarity_positive'] = ''
create_polarities(df)
create_polarities(df2)

In [31]:
df['polarity_negative']

0         [türkei, einwohner, coronavirus, syrien, flagge]
1         [türkei, einwohner, coronavirus, syrien, flagge]
2        [tuerkenkriege, tuerkenbundlilie, tuerkenbund,...
3        [türkei, zurücktreten, gefangene, coronavirus,...
4        [polemik, coronavirus, dance, polenta, polen, ...
                               ...                        
52459                                                     
52460    [treue, erst, euro, wertlos, ruf, von, transla...
52461                      [bei, massa, mond, hohl, kassa]
52462                                                     
52463                                                     
Name: polarity_negative, Length: 52464, dtype: object

In [18]:
df2['tokens_suggestions_cleaned_root']

337      ['geschichte', 'feste', 'anhänger', 'religion'...
340      ['feste', 'verbreitung', 'fasten', 'referat', ...
341      ['geschichte', 'feste', 'verbreitung', 'religi...
343      ['feste', 'verbreitung', 'fasten', 'referat', ...
344      ['argumente', 'zitate', 'definition', 'anfänge...
                               ...                        
27007    ['geschichte', 'sprüche', 'zitate', 'makhachev...
27022    ['geschichte', 'feste', 'steckbrief', 'english...
27027    ['lamm', 'china', 'english', 'definition', 'be...
27033    ['deutschland', 'glaube', 'definition', 'in', ...
27040    ['symbol', 'prayer', 'definition', 'for', '360...
Name: tokens_suggestions_cleaned_root, Length: 64, dtype: object

In [32]:
#write to richtig2.csv and richtig_polarities.csv
df2.to_csv('richtig2.csv', sep=';', index=False, encoding='utf-8')
df.to_csv('richtig_polarities.csv', sep=';', index=False, encoding='utf-8')
df_from_csv2 = pd.read_csv("richtig2.csv", sep=';', encoding='utf-8')